***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [7. Observing Systems](0_introduction.ipynb)  
    * Previous: [7.3 Analogue Electronics](7_3_analogue.ipynb)
    * Next: [7.5 The Primary Beam](7_5_primary_beam.ipynb)

***

Import standard modules:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import HTML 
HTML('../style/course.css') #apply general CSS

In [2]:
HTML('../style/code_toggle.html')

***

## 7.4 数字相关器 Digital Correlators

在前一节的最后，我们讨论了把连续模拟信号转换成离散数字信号的ADC。电压信号一旦变为数字信号，就可以进入计算机内存，利用软件进行计算，对于干涉测量观测来说，就是计算天线的两两相关，得到基线的相关量。多数干涉阵都会产生巨大的数据量，相关算法很关键。因此，往往不会采取把数字信号存下来再进行相关运算的办法，而是利用基于硬件定制的数字信号处理模块（digital signal processing，DSP），如FPGA、ASICs、以及GPU，这种方式的好处是高效，但灵活性受限。对于天线数量较少或者灵活性要求较高的阵列来讲，可以使用软件完成相关运算。最终采取什么方式，取决于工程造价以及工程的时效，但是相关器系统的核心跟本质都是实现DSP运算。

We ended the previous section by discussing the ADC, the analogue component which converts a continuous, analogue signal into a discrete, digitial representation. Once the voltage signal is converted to a digital signal, we can simply record that signal to computer memory, and perform whatever computational operations we desire with software. In the case of interferometric observations, this means computing the correlation between all antenna pairs to produce baseline correlations. Most interferometric arrays produce significant amounts of data, and correlation is a deterministic algorithm. So, instead of recording the digitial signals and then performing the correlations, it is common to build correlators using *digital signal processing* (DSP) modules on custom hardware such as *field programmable gate arrays* (FPGAs), *application specific integrated circuits* (ASICs), and *graphical processing units* (GPUs). Using custom hardware for correlation provides a specialized and power-efficient system, at the cost of limited flexiblity. For arrays with a small number of elements, or for maximum flexibility, software-based correlators can be used. The choice of digital architecture depends on the engineering costs and timescale for an array or facility, but at the core of any correlator system are the same basic DSP operations.

射电天文领域还存在一些其它类型的数字设备，如用于计算单天线自相关的光谱仪，把多个天线合成为一个望远镜的波束合成，记录原始电压信号以备后期集中进行相关处理的VLBI记录器。本节的重点在于干涉阵的口径合成，因此只讨论相关器。

We should note here that there exist other types of digital instruments used in radio astronomy - spectrometers for computing the auto-correlation of a single element, beamformers for adding together elements into a single telescope, VLBI recorders which record the raw voltage signal to be correlated later at a central location. Since this section focuses on aperture synthesis with an interferometric array, we will focus on correlators.

### 7.4.1 FX相关器：van Cittert-Zernike理论的应用 The FX Correlator: Application of the van Cittert-Zernike Theorem

在进行干涉阵中所有天线对的相关计算时，要用到相关器。在[$\S$ 4 &#10142;](../4_Visibility_Space/4_0_introduction.ipynb)，我们用van Cittert-Zernike Theorem证明了，可见度函数等效于两个接收器的相互相干：

A correlator is used to compute the correlations of all antenna pairs in an interferometric array. In [$\S$ 4 &#10142;](../4_Visibility_Space/4_0_introduction.ipynb) we showed that the visibility function is equivalent to the mutual coherence of two spatialy separated receivers, using the van Cittert-Zernike Theorem:

$$
\Gamma_{pq}(u,v,\tau=0) = \int_{-\infty}^{\infty}\int_{-\infty}^{\infty} { I_\nu(l,m) e^{-2\imath\pi(u_{12}l+v_{12}m)}} dl dm =V_{pq}(u,v,0)
$$

这里的互相干函数$\Gamma_{pq}$为

where the mutual coherence function $\Gamma_{pq}$ is 

$$
\Gamma_\text{pq}(u,v,\tau)= \lim_{T \rightarrow \infty} \frac{1}{2T} \int_{-T}^{T} E_p(t)E_{q}^{*}(t-\tau) dt
$$

该函数等效于两信号的相关([$\S$ 2.6.1 &#10142;](../2_Mathematical_Groundwork/2_6_cross_correlation_and_auto_correlation.ipynb))。根据卷积理论([$\S$ 2.7.4, $\S$ 2.7.8 &#10142;](../2_Mathematical_Groundwork/2_7_fourier_theorems.ipynb))，可见度函数为

This function is equivalent to the correlation between two signals ([$\S$ 2.6.1 &#10142;](../2_Mathematical_Groundwork/2_6_cross_correlation_and_auto_correlation.ipynb)). Using the Convolution Theorem ([$\S$ 2.7.4, $\S$ 2.7.8 &#10142;](../2_Mathematical_Groundwork/2_7_fourier_theorems.ipynb)) the visibility function is then computed as

$$
V_{pq}(u,v,0) = \left( \mathscr{F} \left\{ {E_p} \right\} \right)^* \cdot \mathscr{F} \left\{ E_q \right\}
$$

因此，为了计算基线可见度，需要对天线信号做傅立叶变换，算出一个天线信号的复共轭，然后跟另外一个天线信号相乘。注意，这里选择哪一个天线的复共轭并不重要，因为 $V_{pq} = V_{qp}^*$ 或者 $V_{pq}(u,v) = V_{qp}(-u, -v)$。上述操作被叫做“FX”相关，因为对信号先做傅立叶变换（F），然后做复数乘运算（X）。做完乘法运算以后，再在积分时长内求平均，因为相关属于“统计”运算。通过这种长时间求平均的方法，可以提高信噪比，同时减少仪器的总数据量。在描述时间范围的时候，术语“积分时间（integration time）”跟“累积时间（accumulation length）”常常互用。最长积分时间有限制，在实际应用中，通常根据图像域中的“拖尾”和“去相关”效应进行设置。在([$\S$ 2.6.1 &#10142;](../5_Imaging/5_5_widefield_effect.ipynb))讨论过这些效应。针对一些特殊的物理过程非常短暂的科学研究领域，如瞬变事件和脉冲成像，也有用到非常短的积分时间（< 1秒）。FX相关器的总体结构如图7.4.1。

Thus, in order to compute a baseline's visibility, the antenna signals are Fourier transformed, the complex conjugate is taken for one of the antenna signals, and it is multiplied with the other antenna signal. A note: the choice of complex conjugation does not make a difference, *as long as it is consistently recorded*, as $V_{pq} = V_{qp}^*$ or - equivalently - $V_{pq}(u,v) = V_{qp}(-u, -v)$. This operation is known as an 'FX' correlation, as the signals are first Fourier transformed (F) and then complex multiplied (X). After multiplication, the resulting signal is averaged over some *integration time length*, since the correlation is a *statistical* operation. This accumulation improves the signal to noise ratio and reduces the instrument's overall data rate. The terms *integration time* and *accumulation length* are often used interchangibly to describe this time scale. There are limits to the maximum integration time, which are set in practice by an image-domain effect known as *smearing* or *decorrelation*. This effect is discussed in ([$\S$ 2.6.1 &#10142;](../5_Imaging/5_5_widefield_effect.ipynb)). Very short integration times (< 1 second) are used is specific science cases, such as for fast transient events or pulsar imaging where the time scale of the physical process is very short. The general architecture of the FX correlator is shown in Figure 7.4.1.

<img src='figures/fx_diagram-crop.svg' width=90%>

**Figure 7.4.1:** FX相关器结构。来自天线的数字信号通过多相位滤波器（poly-phase filterbank，PFB)进行傅立叶变换，PFB由一个有限脉冲响应（finite impulse response，FIR）滤波器和一个快速傅立叶变换（fast Fourier transform，FFT)构成。然后每组信号在相关三角中用复数乘法和累积（complex multiply and accumulate，CMAC)合成，得到相关矩阵。<a id='fx_diagram-crop'></a><!--\label{fx_diagram-crop}-->
FX Correlator architecture. The digitized signal from each antenna element is Fourier transformed with a poly-phase filterbank (PFB) consisting of a finite impulse response (FIR) filter and a fast Fourier transform (FFT). Each pair of signals is then complex multiplied together in the correlation tringale using a complex multiply and accumulate (CMAC) to produce a correlation matrix. <a id='fx_diagram-crop'></a><!--\label{fx_diagram-crop}-->

有两类相关器：

* *互相关*: 两天线$p$ 和 $q$ 且 $p \neq q$之间的相关，得到uv平面上原点以外的一个采样点。
* *自相关*: 天线自己跟自己相关，相当于一个频谱仪测量。又叫做“零基线”采样，因为它是uv平面上的原点采样。射电干涉测量中一般忽略自相关。图7.4.1中，用相关矩阵右边的矩形表示。

There are two types of correlations a correlator can compute:

* *cross-correlation*: a correlation between antennas $p$ and $q$ such that $p \neq q$ which results in a sampling of the *uv*-plane outside the origin.
* *auto-correlation*: a correlation of an antenna with itself, equivalent to a spectrometer measurement. This is known as a 'zero baseline' sampling, as it is a sample of the origin of the $uv$-plane. The auto-correlations are typically ignored in radio interferometry. In Figure 7.4.1 these are represented by the squares on the far right of the correlation matrix.

我们可以利用干涉阵对$uv$-plane采样从而实现口径合成，这就意味着我们感兴趣的是“互相关”。自相关测量的都是uv平面的原点，这是uv面上的一个特殊点，其值对应的是天空的平均通量，从信号处理的角度来看，是“DC bias” 或“offset”，是傅立叶变换的第一个系数。

With an interferometric array, we are interested in sampling the $uv$-plane to perform aperture synthesis. This means we are interested in the *cross-correlations*. The auto-correlations all measure the origin of the $uv$-plane. This is a special point in the plane, as it measures the average flux of the sky. From a signal processing perspective, this is the 'DC bias or offest' point, and is the first coefficient of the Fourier Transform.

我们需要测量原点的可见度函数，因为我们可以据此得知合成图像的绝对通量大小。然而，这事实上相当困难。由于每个天线产生的噪声不一样，所以，我们计算两天线的互相关时，大量系统噪声会相互抵消。但是，在计算自相关时，因为噪声是自己跟自己相关，所以会留在相关结果中。为了测量原点处的可见度函数，需要设计特殊的天线，增加电子器件，以削弱系统噪声，或者结合使用单碟观测跟干涉测量观测。

We are interested in measuring the visibility function at the origin, as it provides an absolute flux scale to the synthesized image. This turns out to be fairly difficult in practice, however. When we compute a cross-correlation between antenna pairs, much of the system noise conveniently disappears, because each antenna will be generating noise which is incoherent with the other antenna's. When computing the auto-correlation, however, the noise is being correlated with itself - and remains in the correlation. In order to measure the visibility function at the origin, special antennas need to be built in order to mitigate the system noise with additional electronics or by combining interferometric observations with single-dish observations.

一个有$N_{\textrm{antennas}}$面天线的干涉阵，相关数量为

For an $N_{\textrm{antennas}}$ element interferometric array the number of correlations is:

$$
N_{\textrm{corr}} = N_{\textrm{auto corr}} + N_{\textrm{cross corr}} = N_{\textrm{antennas}} + \frac{N_{\textrm{antennas}} (N_{\textrm{antennas}}-1)}{2}
\sim N_{\textrm{antennas}}^2
$$ 

随着天线数$N_{\textrm{antennas}}$的增加，自相关数量以线性增长，而互相关数量以平方增长。当天线数等于或大于10，相比于自相关，互相关数很快主导了计算量。自相关通常用于单个望远镜诊断，或特定的科研场景，如功率谱测量。

As $N_{\textrm{antennas}}$ increases the number of auto-correlations increases linearly while the number of cross-correlations grows quadratically. For $N_{\textrm{antennas}}$ of the order of 10 or more, the number of cross-correlations quickly dominates the number of computations compared to auto-correlations. The auto-correlations are usually computed for to diagnose individual telescopes, or used in special science cases such as power spectrum measurements. 

据历史记载，第一台相关器被称作“滞后相关器lag correlators”，当时还没有使用计算机做快速傅立叶变换，干涉阵列的天线数量不多，观测带宽很窄。相关器给天线增加一系列滞后（lags），然后每个被滞后的信号跟来自参考天线的信号相乘。对每组相关滞后信号积分并保存，然后做傅立叶变换，得到可见度。随着FX相关器的出现，滞后相关器被重新命名为“XF相关器”，因为滞后相关器是先乘（X）后进行傅立叶变换（F）。XF相关器的总体结构见图7.4.2。这种相关器目前已经用的很少，只在少数特殊场合能见到。现代干涉测量阵列，提到相关器的时候，几乎都指的是FX相关器。本节后面的内容也只讨论FX相关器，对滞后相关器感兴趣的读者可以参考 [<cite data-cite='1999ASPC..180.....T'>Chapter 4 of Synthesis Imaging in Radio Astronomy II</cite> &#10548;](http://adsabs.harvard.edu/abs/1999ASPC..180.....T)。

As a historical note, the first correlator designs were called 'lag correlators'. These correlators pre-date the era of fast Fourier transform implementation using computing hardware, and were built in an era when an array consisted of only a few antennas and the observational bandwidth was fractionally small. This correlation was done by adding a series of time delays (lags) to one antenna, and multiplying the signal from each lag with the signal from a reference antenna. Each correlated lag signal was then integrated for a length of time and subsequently recorded. Later, the recorded correlations at different lags were Fourier transformed to produce the visibility. With the development of FX correlators the lag correlator was renamed the XF correlator as the multiplication (X) we performed before the Fourier tranform (F). Their general architecture is shown in Figure 7.4.2. This type of correlator is rarely used today, except in special cases. For modern interferometric arrays, when correlation is mentioned it is almost always done via FX correlation. For the remainder of this section we will only discuss FX correlators, and readers interested in further details on lag correlators are referred to [<cite data-cite='1999ASPC..180.....T'>Chapter 4 of Synthesis Imaging in Radio Astronomy II</cite> &#10548;](http://adsabs.harvard.edu/abs/1999ASPC..180.....T).

<img src='figures/xf_diagram-crop.svg' width=90%>

**Figure 7.4.2:** XF相关器构造。来自每面天线的信号都做了M次滞后，用M-tap lag multiply and accumulator (MAC)把每对天线的滞后信号相乘，保存，然后做傅立叶变换得到相关矩阵。   XF Correlator architecture. M time delays (lags) are applied to the signal from each antenna element. The lagged signals for each pair of antenna elements are then multiplied together with an M-tap lag multiply and accumulator (MAC). These signals are recorded to memory, and the lag-multiplied signals are later Fourier-transformed to produce the elements of the correlation matrix. <a id='instrum:fig:xf_diagram-crop></a><!--\label{instrum:fig:xf_diagram-crop}-->

尽管FX和XF的结构不同，但是产生的结果等价，因为原理都是卷积理论。见图7.4.3.

Thougth the FX and XF architectures are different they produce equivalent results, Figure 7.4.3, as they are both applications of the Convolution Theorem.

<img src='figures/fx_xf_equiv-crop.svg' width=60%>

**Figure 7.4.3**: FX-XF结构。两种设计都属于卷积理论的应用，用于产生等效的可见度。相比之下，FX相关器更为高效。摘自 [<cite data-cite='1999ASPC..180.....T'>Synthesis Imaging in Radio Astronomy II, Chapter 4</cite> &#10548;](http://adsabs.harvard.edu/abs/1999ASPC..180.....T). <a id='instrum:fig:fx_xf_equiv-crop'></a><!--\label{instrum:fig:fx_xf_equiv-crop}-->。FX-XF Correlator architecture equivalence. Both designs can be used to produce the equivalent visibilities, as both are an application of the Convolution Theorem. FX correlators are generally the most computationally efficient, however. Adapted from [<cite data-cite='1999ASPC..180.....T'>Synthesis Imaging in Radio Astronomy II, Chapter 4</cite> &#10548;](http://adsabs.harvard.edu/abs/1999ASPC..180.....T). <a id='instrum:fig:fx_xf_equiv-crop'></a><!--\label{instrum:fig:fx_xf_equiv-crop}-->

### 7.4.2 FX相关器的数字实现 Digital Implementation of an FX Correlator

虽然FX相关器的确切实现取决于仪器需求跟硬件架构，但设计中还是存在一些典型的通用组件。下图展示了一个数字相关器设计的概况。来自天线的信号通过ADC进行数字化，然后通过“F”引擎进行傅立叶变换，得到覆盖数字带宽的多个频道的未积分频谱。除了完成傅立叶变换，“F”引擎包括了一些其它操作，如增益校准、条纹跟踪以及量化。为了实现每组天线信号的互乘跟累积，各频道的处理结果被送往一系列“X”引擎，这种变换通过转弯操作完成。得到的可见度以测量集“Mesurement set”格式保存，其中包括了望远镜跟观测的元数据信息。

The exact implementation of an FX Correlator depends on instrument requirements and hardware architecture, but there is a typical set of components which are included in the design. An overview of a digital correlator design is shown in the figure below. The signal from each antenna is digitized with an ADC and Fourier transformed with 'F' engines. This produces an unaccumulated spectrum made up of frequency channels which cover the digital bandwidth. In the 'F' engine, additional operations such as gain correction, fringe tracking, and quantization can be implemented. To perform the cross multiplication and accumulation for each antenna pair, a subset of these frequency channels are sent to various 'X' engines. This transform is done with a cornerturn operation. The resulting visibilities are recorded to file, often in a 'Measurement Set' which contains further metadata on the observation and synthesized telescope.

<img src='figures/fx_block_diagram.svg' width=80%>

**Figure 7.4.4**: 一个分布式FX相关器的设计框图。来自N面天线的正交极化信号经过数字化处理，进入“F”引擎进行傅立叶变换。通过转弯处理，数据被分割到多个频道，在“X”引擎中完成互乘跟累积。把最后的可见度数据写入文件保存。Block diagram of a distributed FX correlator design. The orthogonal polarization from N antennas are digitized and Fourier transformed in the 'F' engines. A cornerturn operation is applied to segment the data by frequency channels, to be cross multiplied and accumulated in the 'X' engines. The resulting visibilities are then written to file. <a id='instrum:fig:fx_block_diagram'></a><!--\label{instrum:fig:fx_block_diagram}-->

#### 傅立叶变换跟多相滤波 Fourier Transform and Polyphase Filterbanks

F引擎（如下图所示），主要有DSP构成，通过快速傅立叶算法把时域信号转换成频域的频谱。通过模拟系统的带通跟数字化处理，选择部分频段进行相关运算。事实上，我们也可以进行全带宽的相关运算，但是这样的处理只能得到整个频带的信息。如果存在强的窄带RFI，相关结果会被RFI占满，从而丢失天空信号。如果我们只对弱的窄带信号，如谱线，感兴趣，全带宽运算会使得谱线结构淹没在相关结果中。为了获得小段频率范围（称为“子带”或“通道”）的信号，要用到傅立叶变换。在这种情况下，傅立叶变换把宽带时域信号转换成了一系列频域信号（给定采样窗口）。因为时离散采样，所以可以用离散傅立叶变换（Discrete Fourier Transform ，DFT)。

The F-engine, a block diagram shown below, is primarily composed of DSP to Fourier-transform the time-domain signal into a frequency-domain spectrum with a fast Fourier transform algorithm. The bandpass of the analogue system and digitisation process select a range of frequencies to be correlated between antennas. The full band from each antenna can be chosen to be correlated against another antenna, but this would only give information about the entire band. If there is strong narrowband RFI, then any correlation will be dominated by that, and the sky signal will be lost. If we are only interested in a weak narrow signal, such as a spectral line, that line structure will be hidden in the band correlation. In order to gain access to subsets of the frequency range, known as *subbands* or *channels*, a Fourier Transform is employed. In this context, a Fourier Transform will transform a wideband time-domain signal into a set of frequency-domain signals (for a given 'window' of samples). As we are working on discrete samples sampled regularly in time, a *Discrete Fourier Transform (DFT)* can be used.

DFT的计算成本可以简单计算为$\mathcal{O}(N^2)$，即便是较小的$N$值，计算成本也会增长很快，这里$N$为采样（通道）数。针对这一问题的解决方案是采取快速傅立叶变换（Fast Fourier Transform，FFT)，通过规则采样使运算量降低到$\mathcal{O}(N \log N)$。“Cooley-Tukey Radix-2 algorithm”是一个经典的FFT算法。值得注意的是，FFT的规模通常为$2^d$，即256, 512, 1024等等，原因在于$2^d$ 可以使得FFT更为高效，其它规模，即便$<2^d$，也会消耗更长的计算时间。

The naive computational cost of the DFT is $\mathcal{O}(N^2)$, which can grow to be very computationally expensive for relatively small sizes of $N$, where $N$ is the number of samples (or channels). The solution to this computational problem is the *Fast Fourier Transform (FFT)* which is a class of algorithms with use the regularity of the sampling to approach a run time of $\mathcal{O}(N \log N)$. The *Cooley-Tukey Radix-2 algorithm* is "the" classic example of a FFT algorithm. Important note: you will often see FFTs of size $2^d$, i.e. 256, 512, 1024, etc., being used. This is because a size of $2^d$ allows the use of the most efficient FFT. Other FFT sizes, even sizes $<2^d$, will take longer to compute.

要点：做FFT的时候，采样数尽量选择为$2^d$。

This leads to the following **important** point: ***any time you are doing an FFT, try to pick a size which is of the form $2^d$***.


我们继续讨论离散采样时域信号。FFT窗口里的采样数影响到子带的分辨率，对于有$N$个实数值的时域信号，会生成$N/2$个复频率子带。造成2倍信息“损失”的原因在于，FFT变换的输入是实数，输出是复数，因此每个子带有2个值，事实上并没有信息的净损失。窗口采样越多，输出子带的分辨率越高。如果窗口无限宽，则DFT接近于连续傅立叶变换的输出。在另一极端情况下，如果采样窗口为1，则输出跟输入相同。把FFT的输出看成是一系列窄带时域信号，这在DSP领域被称为“滤波器组filterbank”。

Let us close this parenthesis and return to our discretely-sampled time-domain signal. Our choice of how many samples to include in the window in which we apply the FFT affects the subband resolution. For $N$ real-valued time-domain signals, $N/2$ complex frequency subbands are created. The apparent factor of 2 'loss' in information due to the FFT is because the output of the FFT is complex and the input is real, so there are effectively 2 values per frequency subband - there is no net information loss. The more samples are included in the window, the higher the resolution of subband outputs. In the extreme case of an infinite window the DFT approaches the continuous Fourier Transform output. On the other exteme, a window of 1 sample will produce the same output as input. It is often useful to think of the FFT generating a series of narrow-band time domain signals: this is called a *filterbank* in DSP.


这里一直用术语“窗”来描述FFT的采样区域。从一个连续信号中选择出一段进行采样，事实上，您就已经改变了该信号，因为已经对该信号应用了“礼帽函数（矩形函数）”。虽然没有对采样做任何修改，但是，信号的连续性发生了变化。将FFT作用于窗，从卷积理论的角度来看，就是数据的傅立叶变换与礼帽函数的傅立叶变换进行卷积。礼帽函数的傅立叶变换是一个“辛格sinc函数”，定义为$\textrm{sinc}(x) = \frac{\sin x}{x}$。礼帽函数又被称为“窗函数”，事实上它的性能并不怎么好，主要问题在于旁瓣高和滚降慢。除了礼帽函数，还有许多其它窗函数：Hamming，Hann，Blackman，Gaussian等。这些窗函数考虑了子带宽度、旁瓣水平、旁瓣滚降以及锐度。为什么要使用窗函数进行采样，一个简单的例子，如果子带中存在强RFI，同时窗函数的旁瓣高（如礼帽函数），那么从RFI子带中泄漏出的能量会污染到旁边的子带。


I have been using the term windows for blocks of samples which an FFT is applied to. If you take a continuous signal and select out a block of samples, then you have altered the signal: you have effectively applied a *top-hat function* (rectangle) to the signal. This might not seem like an issue since none of the samples have been modified, but the continuity of the signal has been altered. When an FFT is applied to the window, from the convolution theorem, we are convolving the Fourier transform of the data with the Fourier transform of a top-hat function. The Fourier transform of the top-hat function is a *sinc function* which is defined as $\textrm{sinc}(x) = \frac{\sin x}{x}$. The top-hat function is known as a *window function*, and it turns out to be a generally poor one. The primary issue with the top-hat function is that is that there are high 'sidelobes' with a 'slow' roll-off. Many other window functions have been developed: Hamming, Hann, Blackman, Gaussian, etc. These windowing functions have a trade off in the subband width, sidelobe levels, sidelobe roll-off, and sharpness. A simple example as to why you would want to apply a window function to samples is that if there is strong RFI in one subband, and the windowing function has high sidelobes (like a top-hat function), then power will *leak* from the RFI subband to clean neighbour subbands, corrupting it.


为了将窗函数作用于信号，在对信号进行傅立叶变换之前，要对其进行“有限脉冲响应（finite impulse response，FIR)”。FIR滤波器加上FFT，被称为“多相滤波器组（poly-phase filterbank，PFB)”，所有的现代数字后端都采用PFB，而不是简单的FFT。

To apply a window function to the signals, a *finite impulse response* (FIR) filter is applied before the Fourier transform. The FIR filter - together with the FFT - is known as a *poly-phase filterbank (PFB)*, and all modern digital back-ends use this instead of the simple FFT.

<img src='figures/f-engine_diagram.svg' width=90%>

**Figure 7.4.5:** F引擎流程图。DSP的核心是傅立叶变换，把时域电压信号转换成频域能谱。这种变换通常采用快速傅立叶算法，以及一个有限脉冲响应滤波。采用各种信号延迟方法做条纹跟踪，然后进行增益校准跟量化并存储。Block diagram of an F-engine. At the centre of the DSP is a Fourier transform to transform the time-domain voltage signals to a frequency-domain spectrum. This transform is done with a fast Fourier transform algorithm, and is often accompanied with a finite impluse response filter in what is called a polyphase filterbank. Fringe tracking can be applied within the F-engine using various signal delay methods. Gain correction and quantization is applied before the signal is transmitted to cornerturn memory. <a id='instrum:fig:f-engine_diagram'></a>

#### 条纹跟踪相位中心 Fringe-tracking the Phase Centre


观测期间，阵列会根据赤经和赤纬$(\textrm{RA}, \delta)$进行位置跟踪。该$(\textrm{RA}, \delta)$位置为观测的“相位中心”。跟踪是通过“条纹跟踪”或“条纹停止”逻辑实现。利用阵列的天线位置、观测时间以及相位中心，条纹停止逻辑连续不断地更新延迟改正。使用两种类型的延迟：“粗延迟coarse delay”通过整数时间采样延迟输入信号，“细延迟fine delay”通过下级采样进行信号延迟。粗延迟用在傅立叶变换之前，可以看成是增加不同的电缆长度来延迟输入信号。细延迟则是用在傅立叶变换之后，对每个频道进行相位改正。频道不同，相位改正也稍有不同。

During an observation, an array will track a position in right ascension and declination $(\textrm{RA}, \delta)$ on the sky (typically speaking, anyway; there are special cases such as transiting arrays). This $(\textrm{RA}, \delta)$ position is the *phase-centre* of the observation. Tracking is done with *fringe-tracking* or *fringe-stopping* logic. Using the position of the array elements, time of observation, and phase centre, the fringe-stopping logic continuously updates delay corrections. Two types of delays are used: *coarse delay* to delay the incoming signals by integer time samples, and *fine delay* to delay signal by subsamples. The coarse delay is applied before the Fourier transform and can be thought of as physically inserting cables of different lengths to delay the incoming signals. Fine delay is applied after the Fourier transform as a phase correction to each frequnecy channel of the spectra. This phase correction is slightly different for each frequency channel.


天线信号以采样频率$f_{\textrm{samp}}$进行数字化，最大数字频率（奈奎斯特频率）$f_{\textrm{nyquist}} = \frac{f_{\textrm{samp}}}{2}$。采样频率也可以看成是采样时间分辨率$\Delta t_{\textrm{samp}} = \frac{1}{f_{\textrm{samp}}}$，即天空信号以多大时间间隔进行采样。因此，粗延迟只能按采样率进行信号延迟。例如，如果对250MHz的带宽进行奈奎斯特采样，ADC的采样率为500Msps（mega-sampling per second），则采样时间分辨率为$\Delta t_{\textrm{samp}} = 2$纳秒。该时间分辨率也可以看成是物理上的距离分辨率，因为光速为常数$3x10^8$ m/s，则$\Delta d_{\textrm{samp}} = \frac{c}{f_{\textrm{samp}}}$=60cm。这就意味着，即便我们知道了天线位置且精度高于60cm（天线位置精度通常为毫米量级），延迟也不能被正确校准，从而导致天线间信号的去相关。

What occurs with these two different types of delays comes down the fact that the antenna signals have been digitized at some sampling frequency $f_{\textrm{samp}}$ which results in a maximum digital frequency (i.e. a Nyquist frequency) of $f_{\textrm{nyquist}} = \frac{f_{\textrm{samp}}}{2}$. This sampling frequency can also be thought of a sampling time resolution $\Delta t_{\textrm{samp}} = \frac{1}{f_{\textrm{samp}}}$: how often the incoming sky signal is sampled. Therefore, coarse delay can only delay a signal at a resolution of the sampling rate. For example, say we want to Nyquist sample 250 MHz of bandwidth. We use an ADC which samples at 500 Msps (mega-sampling per second). In this system, the sampling time resolution is $\Delta t_{\textrm{samp}} = 2$ nanoseconds. This time resolution can also be seen as a physical distance resolution, since the speed of light is constant $\Delta d_{\textrm{samp}} = \frac{c}{f_{\textrm{samp}}}$. Light travels approximately $3 10^8$ metres per second. Conversion to nanoseconds is done by dividing by $10^9$. In our example system, $\Delta d_{\textrm{samp}} = 60 cm$. This means that, even if we know the location of the antennas (and hence their delay correction) to better than 60 cm resolution - which is very likely as antenna positions are usually known to millimetre accuracy - the delay cannot be properly corrected, which leads to signal decorrelation between antennas.


理想情况下，延迟改正精度至少可以达到已知望远镜位置的分辨率。为此，需要使用细延迟。经过粗延迟改正，分辨率达到了$\Delta t_{\textrm{samp}}$，然后在每个频道上进行相位改正$\phi(\nu)$。相位改正精度为每个频道的波峰宽度。

Ideally, the delays can be corrected up to the resolution of the known telescope positions. To do this, fine delay is used. Once coarse delay is applied to correct delays up to the $\Delta t_{\textrm{samp}}$ resolution, a phase correction $\phi(\nu)$ is added to each frequency channel. The phase correction allows each frequency channel to be delayed up to the wavelegnth of that channel.

为了确定阵列中每个望远镜的延迟，选择一个参考位置。 这通常是天线之一或阵列的中心。 延迟在时间上可以为正，也可以为负，以说明相对于参考位置和天空相位中心的阵列几何形状。

为了确定干涉阵中每台望远镜的延迟，需要先选定一个参考位置，一般选择其中的某面天线或者阵列的几何中心。延迟在时间上可以为正，也可以为负，取决于阵列跟参考位置和天空相位中心的几何位置。

To determine the delays for each telescope in the array, a reference position is chosen. This is typically one of the antennas or the centre of the array. Delays can be both positive and negative in time, to account for the array geometry relative to the reference position and sky phase centre.

这里有一个重要的注意事项，关于边缘停止以跟踪天空中的位置并实际指向菜。 在下一节中，我们将讨论与物理指向碟有关的主光束和偏振。 边缘停止是应用相位和时间延迟校正以消除相位中心处的相位变化（条纹）（即抵消K-琼斯项）的过程。 进行物理指向的望远镜是为了提高灵敏度。 通过设置固定的观察相位中心，需要进行边缘合成来进行孔径合成。 如果没有应用边缘停止，则观察的相位中心会在整个观察过程中沿右提升方向有效地缓慢漂移。

在进行天空位置跟天线物理指向跟踪时，需要注意条纹停止这件事情。主波束与极化跟天线的物理指向密切相关，我们将在下节讨论。条纹停止是应用相位和时间延迟改正去消除相位中心的相位变化（条纹）的过程，即抵消K-琼斯项。物理指向的目的在于提高灵敏度，观测过程中，通过设置一个固定的相位中心，进行条纹停止，实现孔径合成。如果没有条纹停止，观测的相位中心的赤经会发生缓慢漂移。

There is an important note to make here about fringe-stopping to track a position in the sky and physically pointing dishes. We will discuss primary beams and polarization in the next sections, which are related to physically pointing dishes. Fringe-stopping is the process of applying a phase and time delay correction remove phase variation (fringes) at the phase centre (i.e. counteracting the K-jones term). Physically pointing telescopes is done to improve sensitivity. Fringe-stopping is required to do aperture synthesis by setting a fixed phase centre for the observation. If fringe-stopping is not applied, then the phase centre of the observation effectively drifts slowly in right ascension throughout the observation.

#### 增益改正与信号量化 Gain Correction and Signal Quantization

* 例：全动态范围频谱w/量化线数 -> 经过增益改正的频谱w/量化线数

* example: full dynamic range spectrum w/ quantization lines -> flattened with gain correction w/ quantization lines

* 增益：
   * 复数
   * 实时增益校准，RFI分割
   * “展开flatten“带通以便于量化
   

* gain:
    * complex
    * real time gain corrections, RFI clipping
    * 'flatten' the bandpass to prepare for quantization

* 量化
    * 减小数据的采样率
    * 动态范围，高斯信号的有效量化
    * 类似于前一节的ADC
    
* quantization
    * reduce the data rate from the bit growth in the FFT
    * dynamic range, quantization efficiency for a Gaussian signal
    * similar to adc in previous section

In [3]:
# load observed sprectum from analogue section
npzfile = np.load('data/analogue_spectrum.npz')
obsSpectrum = npzfile['arr_0']
ifFreqs = npzfile['arr_1']

fig, axes = plt.subplots(figsize=(8,8))

ax1 = plt.subplot2grid((5,5), (0,0), colspan=4, rowspan=4)
plt.imshow(np.abs(obsSpectrum), aspect='auto')
ax1.get_xaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)

ax2 = plt.subplot2grid((5,5), (4,0), colspan=4, rowspan=1)
plt.plot(ifFreqs/1e9, obsSpectrum[50, :])
plt.plot(ifFreqs/1e9, obsSpectrum[250, :])
plt.plot(ifFreqs/1e9, obsSpectrum[450, :])
plt.xlim(ifFreqs[0]/1e9, ifFreqs[-1]/1e9)
plt.ylabel('Flux')
plt.xlabel('Frequency (GHz)')

ax3 = plt.subplot2grid((5,5), (0,4), colspan=1, rowspan=4)
plt.plot(obsSpectrum[:, 130], np.arange(obsSpectrum.shape[0]))
plt.plot(obsSpectrum[:, 260], np.arange(obsSpectrum.shape[0]))
plt.plot(obsSpectrum[:, 390], np.arange(obsSpectrum.shape[0]))
ax3.get_xaxis().set_visible(False)
ax3.invert_yaxis()
ax3.yaxis.tick_right()
plt.ylabel('Time (s)')
ax3.yaxis.set_label_position('right')

plt.suptitle('Observed Spectrum')

FileNotFoundError: [Errno 2] No such file or directory: 'data/analogue_spectrum.npz'

#### 转弯操作 The Cornerturn Operation

* 从大带宽/短时间区域移动到小带宽/长时间区域
* 转置
* 发送子带数据到不同的x-引擎，进行分布计算

* move from large bandwidth/short time, to small bandwidth/long time blocks of samples
* transpose
* send subsets of the frequency channels to different x-engine to distribute the computation

<img src='figures/cornerturn_diagram.svg' width=50%>

**Figure 7.4.6:** A cornerturn operation is applied to the output of each F-engine. T spectra are written to a block of memory. Then, T samples of each frequency channel are read out to be sent to X-engines for cross multiplication. This operation is equivalent to applying a transpose. <a id='instrum:fig:cornerturn_diagram'></a><!--\label{instrum:fig:cornerturn_diagram}-->

#### Building the Correlation Matrix

* cross-multiply and accumulate (CMAC)
* vector accumulation
* example: waterfall visibilities

<img src='figures/x-engine_diagram.svg' width=60%>

**Figure 7.4.7:** 由7个望远镜构成的阵列（如KAT-7）的相关矩阵。绿色为自相关，蓝色为21个互相关。矩阵中每个单元对应一对天线的复数乘，经过积分生成可见度。Correlation matrix for a 7-telescope array, such as KAT-7. The seven auto-correlations are shown in green, while the 21 cross-correlations are shown in blue. Each cell of the matrix is a complex multiplication of the antenna pairs, and accumulation/averaging of the signal to produce the visibility. <a id='instrum:fig:x-engine_diagram'></a><!--\label{instrum:fig:x-engine_diagram}-->

### 7.4.3 数据产品（Measurement Set）Resulting Data Product (The Measurement Set) <a id='instrum:sec:7_4_3'></a>

* 输出结果：相关矩阵，带元数据的测量集（uvw，时间，观测信息，频率，标记flags）
* 给相关矩阵中的可见度一个uvw坐标和标记flag
* MS参考文件

* resulting output: correlation matrix, measurement set with metadata (uvw, time, observation information, frequency, flags)
* assign each visibility in the correlation matrix a (uvw) position, flag
* reference MS papers

***

* [Outline](../0_Introduction/0_introduction.ipynb)
* [Glossary](../0_Introduction/1_glossary.ipynb)
* [7. Observing Systems](0_introduction.ipynb)  
    * Previous: [7.3 Analogue Electronics](7_3_analogue.ipynb)
    * Next: [7.5 The Primary Beam](7_5_primary_beam.ipynb)

***

<div class=warn><b>Future Additions:</b></div>

* discuss smearing/decorrelation, goes in imaging (5.5?)
* expand correlator history (7.4.1.1)
* show the process of correlation two signals to produce a fringe: two voltage signals of a wideband signal (noise and delayed), fourier transform to produce spectrum, multiply to produce fringe
* section on beamforming, relate to correlation
* FFT, FIR, PFB: FIR window functions (relate to imaging chapter), FFT response with and without FIR (regular/box, hamming, hann, gauss), taps ; relation between spectral resolution and spectra time resolution